In [25]:
import pandas as pd
import pickle
from collections import Counter

import xgboost as xgb
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn import metrics
from sklearn.metrics import confusion_matrix



In [26]:

pd.set_option('display.max_rows', 300)

survey = pd.read_csv('/Users/williamyee/Documents/Data Bootcamp/Classification-Project-DSjobs/Data/kaggle-survey-2017/multi_r_.csv', encoding = 'ISO-8859-1')

list(survey.columns)

survey.JobSatisfaction.value_counts()



survey.head()

survey.EmploymentStatus.unique()

survey.groupby('EmploymentStatus').count()

survey.UniversityImportance.value_counts()

survey.JobSatisfaction.value_counts()




/Users/williamyee/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (32,84,87,88,100,110,117,124,125,128,130,131,165) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


7                          1448
8                          1427
6                           765
9                           677
5                           627
10 - Highly Satisfied       589
3                           358
4                           354
1 - Highly Dissatisfied     167
I prefer not to share       148
2                           117
Name: JobSatisfaction, dtype: int64

In [27]:
conditions = [  survey.JobSatisfaction.isin(['7','8','9','10 - Highly Satisfied']),
                survey.JobSatisfaction.isin(['1 - Highly Dissatisfied','2','3','4','5','6','I prefer not to share'])
             ]
choices = [1,0]
survey['JobSatisfaction'] = np.select(conditions, choices, default=survey.JobSatisfaction)


survey = survey[survey.JobSatisfaction.notna()]

In [28]:
percent_missing = survey.isnull().sum()*100/len(survey)
missing_value_df = pd.DataFrame({'columnname':survey.columns, 'percent_missing':percent_missing})
missing_value_df

,columnname,percent_missing
Unnamed: 0,Unnamed: 0,0.000000
GenderSelect,GenderSelect,0.194698
Country,Country,0.284559
Age,Age,1.258050
EmploymentStatus,EmploymentStatus,0.000000
StudentStatus,StudentStatus,100.000000
LearningDataScience,LearningDataScience,100.000000
CodeWriter,CodeWriter,0.000000
CareerSwitcher,CareerSwitcher,100.000000
CurrentJobTitleSelect,CurrentJobTitleSelect,0.029954


In [29]:
percent_missing = survey.isnull().sum()*100/len(survey)
missing_value_df = pd.DataFrame({'columnname':survey.columns, 'percent_missing':percent_missing})
seventy_five_data = missing_value_df[missing_value_df.percent_missing<40]

seventy_five_data

survey_minimal_missing= survey[list(seventy_five_data.columnname)]

survey_minimal_missing.reset_index(inplace=True)
X = survey_minimal_missing.drop(columns = 'JobSatisfaction',axis =1)
y = survey_minimal_missing['JobSatisfaction']
X

,index,Unnamed: 0,GenderSelect,Country,Age,EmploymentStatus,CodeWriter,CurrentJobTitleSelect,TitleFit,CurrentEmployerType,...,WorkDataVisualizations,WorkInternalVsExternalTools,WorkMLTeamSeatSelect,WorkDataStorage,WorkDataSharing,WorkCodeSharing,RemoteWork,CompensationAmount,CompensationCurrency,SalaryChange
0,0,1,"Non-binary, genderqueer, or gender non-conforming",NaN,NaN,Employed full-time,Yes,DBA/Database Engineer,Fine,Employed by a company that doesn't perform adv...,...,26-50% of projects,Do not know,Standalone Team,Document-oriented (e.g. MongoDB/Elasticsearch)...,"Company Developed Platform,I don't typically s...","Mercurial,Subversion,Other",Always,NaN,NaN,I am not currently employed
1,3,4,Male,United States,56.0,"Independent contractor, freelancer, or self-em...",Yes,Operations Research Practitioner,Poorly,Self-employed,...,100% of projects,Entirely internal,Standalone Team,"Column-oriented relational (e.g. KDB/MariaDB),...","Company Developed Platform,Email",Generic cloud file sharing software (Dropbox/B...,NaN,250000.0,USD,Has increased 20% or more
2,4,5,Male,Taiwan,38.0,Employed full-time,Yes,Computer Scientist,Fine,Employed by a company that doesn't perform adv...,...,10-25% of projects,Approximately half internal and half external,Business Department,Flat files not in a database or cache (e.g. CS...,Company Developed Platform,Git,Rarely,NaN,NaN,I do not want to share information about my sa...
3,5,6,Male,Brazil,46.0,Employed full-time,Yes,Data Scientist,Fine,Employed by college or university,...,76-99% of projects,More internal than external,Other,"Column-oriented relational (e.g. KDB/MariaDB),...",I don't typically share data,"Bitbucket,Git,Other",Sometimes,NaN,NaN,I do not want to share information about my sa...
4,6,7,Male,United States,35.0,Employed full-time,Yes,Computer Scientist,Fine,Employed by government,...,26-50% of projects,Entirely internal,IT Department,Flat files not in a database or cache (e.g. CS...,"Company Developed Platform,Email","Bitbucket,Git",NaN,NaN,NaN,NaN
5,8,9,Female,Australia,43.0,Employed full-time,Yes,Business Analyst,Fine,Employed by non-profit or NGO,...,Less than 10% of projects,Entirely internal,IT Department,Row-oriented relational (e.g. MySQL/Microsoft ...,"I don't typically share data,Share Drive/Share...",Generic non-cloud file sharing software (Email...,Rarely,80000.0,AUD,Has stayed about the same (has not increased o...
6,9,10,Male,Russia,33.0,Employed full-time,Yes,Software Developer/Software Engineer,Fine,Employed by a company that doesn't perform adv...,...,Less than 10% of projects,Do not know,IT Department,Document-oriented (e.g. MongoDB/Elasticsearch)...,I don't typically share data,"Bitbucket,Git",Rarely,1200000.0,RUB,Has increased between 6% and 19%
7,11,12,Male,India,27.0,Employed full-time,Yes,Data Scientist,Fine,Employed by professional services/consulting f...,...,26-50% of projects,Entirely internal,Standalone Team,Document-oriented (e.g. MongoDB/Elasticsearch)...,Company Developed Platform,Bitbucket,Rarely,95000.0,INR,Has increased 20% or more
8,14,15,Male,Taiwan,26.0,Employed full-time,Yes,Software Developer/Software Engineer,Fine,Employed by professional services/consulting firm,...,26-50% of projects,Do not know,IT Department,Flat files not in a database or cache (e.g. CS...,"Email,Share Drive/SharePoint",Git,Sometimes,1100000.0,TWD,Has stayed about the same (has not increased o...
9,15,16,Male,United States,58.0,"Independent contractor, freelancer, or self-em...",Yes,DBA/Database Engineer,Poorly,Employed by professional services/consulting firm,...,26-50% of projects,Do not know,Standalone Team,Document-oriented (e.g. MongoDB/Elasticsearch)...,"Company Developed Platform,Share Drive/SharePoint","Git,Subversion",Most of the time,120000.0,NaN,Has decreased between 6% and 19%


In [30]:
X_cat = X.select_dtypes(include='object')
X_cat = pd.DataFrame(X_cat,dtype ='str')
imp_freq = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
X_cat_imputed = imp_freq.fit_transform(X_cat)
X_cat_imputed = pd.DataFrame(data = X_cat_imputed)

OE = OneHotEncoder()
X_cat_imputed = OE.fit_transform(X_cat_imputed)
X_cat_imputed = pd.DataFrame(data = X_cat_imputed)


X_num = X.select_dtypes(include='float64')
imp_freq = SimpleImputer(missing_values=np.nan, strategy='median')
X_num_imputed = imp_freq.fit_transform(X_num)
X_num_imputed = pd.DataFrame(data= X_num_imputed, columns = X_num.columns)

scale = StandardScaler()
X_num_imputed = scale.fit_transform(X_num_imputed)
X_num_imputed = pd.DataFrame(data= X_num_imputed, columns = X_num.columns)


X_imputed = pd.concat([X_cat_imputed,X_num_imputed],axis =1)



In [31]:
X, X_test, y, y_test = train_test_split(X_imputed, y, test_size=.2, random_state=100)



In [9]:
from sklearn.svm import LinearSVC
from sklearn.svm import SVC